### Решение задания №3. Поиск поставщиков от команды Ezee

Содержание ноутбука:

1. Универсальный модуль парсинга номенклатуры
2. Алгоритм поиска площадок
3. ML модель верификации площадок
4. Алгоритм поиска поставщиков
5. Процедура обогащения профиля поставщика доп. данными
6. ML - модель приоретизации поставщика 

***

In [3]:
# Импорт необходимых библиотек и прочие настройки
import re
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
pd.options.mode.chained_assignment = None

PARSE_NOM_RESULT_FILE = "~nom_parse_result.json"

GET_INN_URL = 'https://yandex.ru/search/?text='
EGRUL_URL = 'https://egrul.itsoft.ru/'
REPUTATION_URL = 'https://vbankcenter.ru/contragent/'

headers = {'user-agent': "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15",
           'accept-encoding': 'gzip, deflate, br',
           'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
           'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           'connection': 'keep-alive',
           'Sec-Fetch-Dest': 'document',
           'Sec-Fetch-Mode': 'navigate',
           'Sec-Fetch-Site': 'none',
           'Sec-Fetch-User': '?1',
           'Upgrade-Insecure-Requests': '1'}

***

### Универсальный модуль парсинга номенклатуры

Модуль преднозначен для получения наименования, характеристик и т.д. из строки-обозначения номенклтуры, представляемой Заказчиком. На вход модуля поступает Pandas DataFrame с полем, в котором содержится строка-обозначение конкретной номенклатурной позиции, после обработки возвращается датафрейм с заполненными полями "Наименование", "Стандарт", "Чистое наименование" и "Характеристики".

In [4]:
def universalNomenclatureParcer(NomenclatureDataFrame):
    # Подготовка шаблонов для разбора строк:
    #   Шаблон для "стандартов":
    patGOST = re.compile(
        r'(ГОСТ\s?Р?\s?(?:ИСО)?\s?\d*[\/]?\d*\s?(?:DIN)?\s?\d*[\/]?\d*)')
    #   Шаблон для метрических параметров:
    patMetricSize = re.compile(r'[мМ]\d{1,}[хХ]?\d{1,}')
    #   Шаблон для текстового описания:
    patWords = re.compile(r'((?:[A-ZА-Я]{1,})?[а-яa-z]{2,})')

    
    def compileFromValues(values):
        # Собирает строки из списка элементов
        result = None
        if values.lastindex > 0:
            prev = ''
            if result:
                prev = result+' '
            result = prev + values[0]
        return result

    def clearFromNone(value):
        return value[5:]

    NomenclatureDataFrame['Стандарт'] = None

    # заполняем поле "Стандарт"
    for i in range(len(NomenclatureDataFrame)):
        gosts = re.finditer(patGOST, NomenclatureDataFrame['Наименование'][i])
        for gost in gosts:
            NomenclatureDataFrame['Стандарт'][i] = str(
                NomenclatureDataFrame['Стандарт'][i]) + ', '+compileFromValues(gost)
            NomenclatureDataFrame['Стандарт'][i] = clearFromNone(NomenclatureDataFrame['Стандарт'][i])
            if NomenclatureDataFrame['Стандарт'][i][0].isdigit() or NomenclatureDataFrame['Стандарт'][i][2] == " ":
                NomenclatureDataFrame['Стандарт'][i] = 'ГОСТ '+NomenclatureDataFrame['Стандарт'][i]


    # заполняем поле "Чистое наименование"
    NomenclatureDataFrame['Чистое наименование'] = None
    NomenclatureDataFrame['Характеристики'] = None
    for i in range(len(NomenclatureDataFrame)):
        # обрабатываем первый тип: "Наименование написано вот так"
        words = re.finditer(patWords, str(NomenclatureDataFrame['Наименование'][i]).replace(
            NomenclatureDataFrame['Стандарт'][i], ""))
        # собираем строку
        for word in words:
            NomenclatureDataFrame['Чистое наименование'][i] = str(
                NomenclatureDataFrame['Чистое наименование'][i])+" " + compileFromValues(word)
        # если строка собралась, нормализируем её, удаляя лишний "мусор"
        if NomenclatureDataFrame['Чистое наименование'][i]:
            NomenclatureDataFrame['Чистое наименование'][i] = clearFromNone(
                NomenclatureDataFrame['Чистое наименование'][i])

            NomenclatureDataFrame['Характеристики'][i] = str(NomenclatureDataFrame['Наименование'][i][1:]).replace(
                NomenclatureDataFrame['Стандарт'][i], "").replace(NomenclatureDataFrame['Чистое наименование'][i], "")

        else:
            raw_str = NomenclatureDataFrame['Наименование'][i].split(';')
            NomenclatureDataFrame['Чистое наименование'][i] = raw_str[0]

    return NomenclatureDataFrame

Пример работы модуля на примере представленных Заказчиком данны (task3/poiskpostav_v1.xlsx):

In [5]:
input_data = pd.read_excel('task3/poiskpostav_v1.xlsx')
input_data

,№,Наименование
0,1,* Манжета М50х70 ГОСТ 22704
1,2,* Манжета М65х90 ГОСТ 22704
2,3,* Манжета М220х250 ГОСТ 22704
3,4,* Манжета М60х80 ГОСТ 22704
4,5,* Манжета резиновая армированная для валов 1.2...
5,6,* Пластина техническая 2Н-I-ТМКЩ-С-4 ГОСТ 7338
6,7,"* Рукав всасывающий В-1-75-У рабочий вакуум 0,..."
7,8,* Рукав с текстильным каркасом В(II)-16-25-38-...
8,9,* Пластина техническая 1Н-I-ТМКЩ-С-5 ГОСТ 7338
9,10,* Манжета резиновая армированная для валов 1.2...


In [6]:
universalNomenclatureParcer(input_data)

,№,Наименование,Стандарт,Чистое наименование,Характеристики
0,1,* Манжета М50х70 ГОСТ 22704,ГОСТ 22704,Манжета,М50х70
1,2,* Манжета М65х90 ГОСТ 22704,ГОСТ 22704,Манжета,М65х90
2,3,* Манжета М220х250 ГОСТ 22704,ГОСТ 22704,Манжета,М220х250
3,4,* Манжета М60х80 ГОСТ 22704,ГОСТ 22704,Манжета,М60х80
4,5,* Манжета резиновая армированная для валов 1.2...,ГОСТ 8752,Манжета резиновая армированная для валов,1.2-120х150х12-1
5,6,* Пластина техническая 2Н-I-ТМКЩ-С-4 ГОСТ 7338,ГОСТ 7338,Пластина техническая,2Н-I-ТМКЩ-С-4
6,7,"* Рукав всасывающий В-1-75-У рабочий вакуум 0,...",ГОСТ 5398,Рукав всасывающий рабочий вакуум,"Рукав всасывающий В-1-75-У рабочий вакуум 0,0..."
7,8,* Рукав с текстильным каркасом В(II)-16-25-38-...,ГОСТ 18698,Рукав текстильным каркасом,Рукав с текстильным каркасом В(II)-16-25-38-ХЛ
8,9,* Пластина техническая 1Н-I-ТМКЩ-С-5 ГОСТ 7338,ГОСТ 7338,Пластина техническая,1Н-I-ТМКЩ-С-5
9,10,* Манжета резиновая армированная для валов 1.2...,ГОСТ 8752,Манжета резиновая армированная для валов,1.2-90х120х12-1


***
### Алгоритм поиска площадок

Учитывая возможную сепецифику запросов, а также предполагаемый рынок закупок, в качестве инструмента получения данных использована поисковая система Яндекс. Модуль получает номенклатурную позицию (запрос) и возращает список пар: "Ссылка на сайт" - "Описание поисковой выдачи".

In [108]:
def getTopFromYandex(request_str):
    # Получаем наменклатурную позицию и возвращаем лист пар: URL = описание
    result_url = []
    result_description = []
    result = []
    SEARCH_URL = "https://yandex.ru/search/direct?text="
    SEARCH_URL_POSTFIX = "&filters_docs=direct_cm"
    request_str = "купить "+request_str[1:]

    raw_result = requests.get(
        SEARCH_URL+request_str+SEARCH_URL_POSTFIX, headers=headers).text

    parsed_result = bs(raw_result, features='lxml')

    cards = parsed_result.find_all('li', {'class': "serp-item desktop-card"})

    for card in cards:
        links = card.find_all('a', {'target': "_blank"})
        for link in links:
            result_url.append(link.get('href'))
        descriptions = card.find_all('span', {'role': 'text'})

        for description in descriptions:
            result_description.append(description.text)

    for _ in range(min(len(result_url), len(result_description))):
        result.append([result_url[_], result_description[_]])

    return result

Пример работы модуля на примере представленных Заказчиком данны (task3/poiskpostav_v1.xlsx) с выдачей результата в виде Pandas DataFrame с полями "Номенкатурное наименование" и "Ссылка на страницу поставщика"

In [109]:
# Пример парсинга имеющейся номенклатуры
# результат выгружается в джейсон curdump.txt
# просмотреть содержимое можно здесь -> http://jsonviewer.stack.hu

res = dict()
for i in range(len(input_data)):
    res[input_data['Наименование'][i]] = getTopFromYandex(
        input_data['Наименование'][i])

res_json = json.dumps(res)
with open(PARSE_NOM_RESULT_FILE, 'w') as res_file:
    res_file.write(res_json)

# Нормализация результатов парсинга номенклатуры

with open(PARSE_NOM_RESULT_FILE, 'r') as jdondatafile:
    jdata = json.load(jdondatafile)

items_urls_descriptions = dict()

tmplUrl = re.compile(r'http[s]?://[^/]*')

for item in jdata.keys():
    items_urls_descriptions[item] = dict()
    for field in jdata[item]:
        if re.findall(tmplUrl, field[0]):
            host_url = re.findall(tmplUrl, field[0])[0]
            if "yabs." in host_url:
                continue
            items_urls_descriptions[item][host_url] = ""

nomenclatureSuppliersLinks = pd.DataFrame(columns=["Номенклатура", "Ссылка на сайт поставщика"])

for item in items_urls_descriptions.keys():
    for link in items_urls_descriptions[item]:
        new_line = {"Номенклатура":item, "Ссылка на сайт поставщика":link}
        nomenclatureSuppliersLinks = nomenclatureSuppliersLinks.append(new_line, ignore_index=True)

nomenclatureSuppliersLinks

,Номенклатура,Ссылка на сайт поставщика
0,* Манжета М50х70 ГОСТ 22704,https://seal-is.com
1,* Манжета М50х70 ГОСТ 22704,https://ms-74.ru
2,* Манжета М50х70 ГОСТ 22704,https://kran-master74.ru
3,* Манжета М50х70 ГОСТ 22704,https://msk.rost-holding.ru
4,* Манжета М50х70 ГОСТ 22704,https://spb-rezina.ru
...,...,...
133,* Гайка шестигранная M42х3.8 покрытие цинковое...,https://market.yandex.ru
134,* БОЛТ; СТАНДАРТ ГОСТ Р ИСО4014 DIN931 (ГОСТ77...,https://market.yandex.ru
135,* БОЛТ; СТАНДАРТ ГОСТ Р ИСО4014 DIN931 (ГОСТ77...,https://www.Super-krepeg.ru
136,* БОЛТ; СТАНДАРТ ГОСТ Р ИСО4014 DIN931 (ГОСТ77...,https://www.vseinstrumenti.ru


***
ML модель верификации площадок

!!! добавить данные в блок !!!

***

Алгоритм поиска поставщиков

Для улучшения качества выдачи результата, разумно поиск поставщиков проводить на специализированных площадках (т.н. маркетплейсах), для этого, учитывая их конечное число, предлагается модуль поиска поставщиков конкретной номенклатурной позиции. Модуль реализован в виде интерфейса, принимающего на вход номенклатурную позицию (запрос) и возращает Pandas Dataframe 

In [110]:
#!!! добавить данные в блок !!!

***
Процедура обогащения профиля поставщика доп. данными

Модуль получает на вход ИНН организации, после чего произвводит обогащение данными, необходимыми для работы алгоритма приоретизации поставщика (размер уставного капитала, прибыль/убыток до налогообложения, прибыль/убыток от продаж) и возвращает Pandas Dataframe с соответствующими полями.

In [111]:
def addDataToSeller(seller_inn):
    result = pd.DataFrame(columns=['Наименование', 'ИНН', 'Уставной капитал', 'Основной ОКВЭД',
                                   'Описание ОКВЭД', 'Код системы налогообложения', 'Статус ЮЛ', 'Дата назначения статуса'])

    full_caption = 'нет данных'
    ust_capitall = 'нет данных'
    main_okved_code = 'нет данных'
    main_okved_descr = 'нет данных'
    tax_system = 'нет данных'
    status_desc = 'нет данных'
    status_date = 'нет данных'

    req_ansv = requests.get(EGRUL_URL+seller_inn+'.json')
    if req_ansv.status_code != 200:
        return
    seller_nalog_data = json.loads(req_ansv.text)
    if 'СвЮЛ' in seller_nalog_data.keys():
        if 'СвНаимЮЛ' in seller_nalog_data['СвЮЛ']:
            full_caption = seller_nalog_data['СвЮЛ']['СвНаимЮЛ']['@attributes']['НаимЮЛПолн']
        if 'СвУстКап' in seller_nalog_data['СвЮЛ']:
            ust_capitall = seller_nalog_data['СвЮЛ']['СвУстКап']['@attributes']['СумКап']
        if 'СвОКВЭД' in seller_nalog_data['СвЮЛ']:
            main_okved_code = seller_nalog_data['СвЮЛ']['СвОКВЭД']['СвОКВЭДОсн']['@attributes']['КодОКВЭД']
            main_okved_descr = seller_nalog_data['СвЮЛ']['СвОКВЭД']['СвОКВЭДОсн']['@attributes']['НаимОКВЭД']
        if 'СвСтатус' in seller_nalog_data['СвЮЛ']:
            status_desc = seller_nalog_data['СвЮЛ']['СвСтатус']['СвСтатус']['@attributes']['НаимСтатусЮЛ']
            status_date = seller_nalog_data['СвЮЛ']['СвСтатус']['ГРНДата']['@attributes']['ДатаЗаписи']
    if 'fin' in seller_nalog_data.keys():
        if 'tax_systems' in seller_nalog_data['fin']:
            tax_system = seller_nalog_data['fin']['tax_systems']['@attributes']['tax_system']

    new_line = {'Наименование': full_caption,
                'ИНН': seller_inn,
                'Уставной капитал': ust_capitall,
                'Основной ОКВЭД': main_okved_code,
                'Описание ОКВЭД': main_okved_descr,
                'Код системы налогообложения': tax_system,
                'Статус ЮЛ': status_desc,
                'Дата назначения статуса': status_date}

    result = result.append(new_line, ignore_index=True)

    return result


Пример обогощения данных на основании запроса по ИНН 3528000597 (ПАО СЕВЕРСТАЛЬ)

In [112]:
addDataToSeller('3528000597')

,Наименование,ИНН,Уставной капитал,Основной ОКВЭД,Описание ОКВЭД,Код системы налогообложения,Статус ЮЛ,Дата назначения статуса
0,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СЕВЕРСТАЛЬ""",3528000597,8377186.6,24.1,"Производство чугуна, стали и ферросплавов",нет данных,нет данных,нет данных


***
ML - модель приоретизации поставщика 

!!! добавить данные в блок !!!
